In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [11]:
model_path = "/Users/trinakarmakar/anaconda2/data/fastai/imdb/models"

In [12]:
#Loading imdb datasets, that has movie reviews and sentiments of those reviews, keras has some helper with the datasets.
from keras.datasets import imdb
idx = imdb.get_word_index()

This is the word list

In [13]:
idx_arr = sorted(idx, key = idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [14]:
path = get_file('imdb_full.pk1', origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
               md5_hash = 'd091312047c43cf9e4e38fef92437263',
               )
filename = 'imdb_full.pk1'
f = open(path, 'rb')
(X_train, y_train), (X_test, y_test) = pickle.load(f)

In [15]:
len(X_train)

25000

You see the word has been replaced by IDs and IDs can be seen in idx2word

In [16]:
idx2word = {v: k for k, v in idx.iteritems()}

In [17]:
', '.join(map(str, X_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [18]:
#The first word of the first review is 23022, The word is
idx2word[23022]

'bromwell'

In [19]:
' '.join([idx2word[o] for o in X_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [20]:
#Reduce vocab size by setting rare words as max index.
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in X_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in X_test]

In [27]:
#Look at the distribution of sentence lengths
lens = np.array(map(len, trn))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

In [23]:
#Pad with zero or truncate sentences to make it unifrom length
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

In [24]:
#This produces really nice rectangular word bank - of size 500 less than this is zero padded and more is truncated
trn.shape

(25000, 500)

#Create simple models - 

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [30]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [31]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 16000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           1600100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [34]:
model.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 22s - loss: 0.4623 - acc: 0.7533 - val_loss: 0.2920 - val_acc: 0.8770  128/25000 [..............................] - ETA: 25s - loss: 0.7357 - acc: 0.5312 - ETA: 20s - lo

The [stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 0.883. So we're short of that, but on the right track.

A CNN is likely to work better because it is designed to work better on ordered data - We need CNN 1d as sequence of words are 1D.

In [67]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [68]:
conv1.compile(loss = 'binary_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])

In [69]:
conv1.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 198s - loss: 0.5644 - acc: 0.7162 - val_loss: 0.3312 - val_acc: 0.8538

In [70]:
conv1.compile(loss = 'binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [71]:
conv1.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 197s - loss: 0.3039 - acc: 0.8734 - val_loss: 0.2857 - val_acc: 0.8803

In [82]:
%cd $model_path
conv1.save_weights('conv1.h5')
conv1.load_weights('conv1.h5')

/Users/trinakarmakar/anaconda2/data/fastai/imdb/models


In this section we will use CNN but with pre trained embeddings.

In [5]:
#Pre trained vectors: 
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [7]:
vecs, words, wordidx = load_vectors('/Users/trinakarmakar/anaconda2/data/fastai/glove/results/6B.50d')

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

We create functional API to create multiple CNN and then concatenate them.

In [8]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [21]:
emb = create_emb()

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [25]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [33]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [34]:
model.layers[0].trainable=True

In [35]:
model.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 229s - loss: 0.4246 - acc: 0.8052 - val_loss: 0.3039 - val_acc: 0.8899

In [36]:
model.layers[0].trainable=True

In [37]:
model.optimizer.lr=1e-4

In [38]:
model.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 227s - loss: 0.2754 - acc: 0.8881 - val_loss: 0.2720 - val_acc: 0.8873

In [39]:
model.save_weights(model_path+'glove50.h5')

Multi-Size CNN:

In [40]:
from keras.layers import Merge

In [41]:
graph_in = Input ((vocab_size, 50))
convs = []
for fsz in range(3,6):
    x = Convolution1D(64, fsz, border_mode='same', activation='relu')(graph_in)
    x = MaxPooling1D()(x)
    x = Flatten()(x)
    convs.append(x)
out = Merge(mode='concat')(convs)
graph = Model(graph_in, out)

In [42]:
emb = create_emb()

In [43]:
model = Sequential ([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [44]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [45]:
model.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 473s - loss: 0.4880 - acc: 0.7478 - val_loss: 0.2969 - val_acc: 0.8838

Interestingly, I found that in this case I got best results when I started the embedding layer as being trainable, and then set it to non-trainable after a couple of epochs. I have no idea why!

In [46]:
model.layers[0].trainable=False

In [47]:
model.optimizer.lr=1e-5

In [49]:
model.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 574s - loss: 0.2871 - acc: 0.8830 - val_loss: 0.2865 - val_acc: 0.8798

This more complex architecture has given us another boost in accuracy.

In [ ]:
LSTM:

In [50]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              W_regularizer=l2(1e-6), dropout=0.2),
    LSTM(100, consume_less='cpu'),
    Dense(1, activation='softmax')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 500, 32)       160000      embedding_input_3[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 100)           53200       embedding_3[0][0]                
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 1)             101         lstm_1[0][0]                     
Total params: 213301
____________________________________________________________________________________________________


In [ ]:
model.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 1601s - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000